
# H-EEWI (Hierarchical Energy–Entropy Wear Index) — Step-by-Step Notebook

This notebook walks you through a **divide & conquer** implementation for the PHM 2010 Milling dataset:

**What you'll do:**
1. Configure paths & parameters
2. Load data (per-cutter) and sanity-check shapes
3. Window the signals (robust to short series)
4. Compute per-band EEWI and fuse to H-EEWI
5. (Optional) Monotone smoothing of H-EEWI vs time
6. Learn global monotone mappings: `H-EEWI → wear fraction s → RUL`
7. Evaluate LOTO (leave-one-tool-out) folds over `TRAIN_CUTTERS`
8. Train on all train cutters, infer on `TEST_CUTTERS`
9. Add **bootstrap**-based uncertainty (two options)

> **Tip:** Run each cell sequentially. Edit the `BASE` path in the Config cell to match your data.


## 1) Imports

In [ ]:

# If something is missing in your environment, uncomment the line below to install.
# !pip install numpy pandas scipy scikit-learn tqdm matplotlib

from __future__ import annotations

import os, glob, math, json
from dataclasses import dataclass, asdict
from typing import List, Tuple, Dict, Optional

import numpy as np
import pandas as pd
from scipy.signal import welch
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm
import matplotlib.pyplot as plt

# Make plots render inline
%matplotlib inline


## 2) Configuration

In [ ]:

@dataclass
class Config:
    # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    # EDIT THIS PATH to your PHM 2010 Milling base directory:
    BASE: str = r"E:\Collaboration Work\With Farooq\phm dataset\PHM Challange 2010 Milling"
    # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

    TRAIN_CUTTERS: Tuple[str, ...] = ("c1", "c4", "c6")
    TEST_CUTTERS: Tuple[str, ...]  = ("c2", "c3", "c5")

    # Sampling & windowing
    fs: float = 50000.0      # Hz
    window_sec: float = 0.1  # 100 ms windows
    hop_sec: float = 0.05    # 50% overlap

    # Frequency bands (adapt to your sensors)
    bands: Tuple[Tuple[float, float], ...] = (
        (500.0, 2000.0),    # Low
        (2000.0, 6000.0),   # Mid
        (6000.0, 20000.0),  # High
    )

    # EEWI mix between energy and entropy: alpha * log(E) + (1-alpha) * H
    alpha_energy: float = 0.6

    # Fusion weights; if None, we learn simple convex weights on a small grid
    fuse_weights: Optional[Tuple[float, float, float]] = None

    # Bootstrap settings
    bootstrap_B: int = 150
    entropy_bootstrap_sigma: float = 0.05  # multiplicative lognormal noise on PSD bins
    window_bootstrap: bool = True          # also do window-level bootstrap

    # Smoothing / monotonic post-projection for H-EEWI curve per tool
    isotonic_smooth_heewi: bool = True

    # Output directory for figures & JSON
    outdir: str = "./heewi_outputs"

CFG = Config()
CFG


## 3) Small utilities

In [ ]:

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def normalize_prob(p: np.ndarray, axis: int = -1, eps: float = 1e-12) -> np.ndarray:
    s = p.sum(axis=axis, keepdims=True)
    return p / np.maximum(s, eps)

def spectral_entropy(psd_band: np.ndarray, axis: int = -1, eps: float = 1e-12) -> np.ndarray:
    """Shannon spectral entropy (ln), normalized to [0,1] by dividing by log(N)."""
    p = normalize_prob(psd_band, axis=axis)
    H = -(p * np.log(np.maximum(p, eps))).sum(axis=axis)
    H_norm = H / np.log(psd_band.shape[axis])
    return H_norm

def band_mask(freqs: np.ndarray, f_lo: float, f_hi: float) -> np.ndarray:
    return (freqs >= f_lo) & (freqs < f_hi)

def compute_welch_psd(x: np.ndarray, fs: float, nperseg: int) -> Tuple[np.ndarray, np.ndarray]:
    freqs, Pxx = welch(x, fs=fs, nperseg=nperseg, noverlap=nperseg//2,
                       detrend='constant', return_onesided=True, scaling='density')
    return freqs, Pxx

def aggregate_multichannel_psd(X: np.ndarray, fs: float, nperseg: int) -> Tuple[np.ndarray, np.ndarray]:
    """Aggregate PSD across channels by summing spectra."""
    freqs, psd0 = compute_welch_psd(X[:, 0], fs, nperseg)
    psd_sum = psd0.copy()
    for c in range(1, X.shape[1]):
        _, psd_c = compute_welch_psd(X[:, c], fs, nperseg)
        psd_sum += psd_c
    return freqs, psd_sum

def eewi_from_psd(freqs: np.ndarray, psd: np.ndarray,
                  bands: List[Tuple[float, float]], alpha: float) -> np.ndarray:
    vals = []
    for (f_lo, f_hi) in bands:
        m = band_mask(freqs, f_lo, f_hi)
        if not np.any(m):
            vals.append(np.nan); continue
        psd_band = psd[m]
        E = np.sum(psd_band)
        H = spectral_entropy(psd_band[None, :], axis=-1)[0]
        vals.append(alpha * np.log(max(E, 1e-12)) + (1.0 - alpha) * H)
    return np.array(vals)

def fuse_hierarchical(eewi_per_band: np.ndarray, weights: Optional[np.ndarray] = None) -> float:
    x = eewi_per_band.copy()
    finite = np.isfinite(x)
    if not np.any(finite):
        return np.nan
    x = x[finite]
    if weights is None:
        w = np.ones_like(x) / len(x)
    else:
        w = weights[finite]
        w = np.maximum(w, 0)
        w = w / w.sum() if w.sum() > 0 else np.ones_like(x) / len(x)
    return float(np.dot(w, x))


## 4) Dataset loader (adapt to your files)

In [ ]:

class PHM2010Loader:
    def __init__(self, base: str, channel_cols: Optional[List[str]] = None):
        self.base = base
        self.channel_cols = channel_cols  # e.g., ["AE", "Vx", "Vy", "Fx", "Fy", "Fz"]

    def list_files(self, cutter: str) -> List[str]:
        pattern = os.path.join(self.base, cutter, "*.csv")
        files = sorted(glob.glob(pattern))
        if not files:
            pattern = os.path.join(self.base, cutter, "**", "*.csv")
            files = sorted(glob.glob(pattern, recursive=True))
        return files

    def load_cutter(self, cutter: str) -> pd.DataFrame:
        files = self.list_files(cutter)
        if not files:
            raise FileNotFoundError(f"No CSV files found for cutter {cutter} under {self.base}")
        dfs = [pd.read_csv(fp) for fp in files]
        df_all = pd.concat(dfs, axis=0, ignore_index=True)
        if self.channel_cols is not None:
            missing = [c for c in self.channel_cols if c not in df_all.columns]
            if missing:
                raise KeyError(f"Missing channels in data: {missing}")
            return df_all[self.channel_cols]
        return df_all.select_dtypes(include=[np.number])  # default: all numeric

# Quick listing to confirm files exist
loader = PHM2010Loader(CFG.BASE, channel_cols=None)  # set your channel list if needed
for c in CFG.TRAIN_CUTTERS + CFG.TEST_CUTTERS:
    print(c, len(loader.list_files(c)), "file(s)")


## 5) Windowing (robust to short series)

In [ ]:

@dataclass
class Windows:
    X: np.ndarray        # [W, T, C]
    t_index: np.ndarray  # [W]

def make_windows(X: np.ndarray, fs: float, window_sec: float, hop_sec: float) -> Windows:
    """Slice multichannel signal X [N, C] into overlapping windows.
    Guarantees at least one window by zero-padding when N < T.
    """
    N, C = X.shape
    T = int(round(window_sec * fs))
    H = int(round(hop_sec * fs))
    if T <= 0 or H <= 0:
        raise ValueError("window_sec and hop_sec must be positive")

    if N < T:
        Xp = np.zeros((T, C), dtype=float)
        Xp[:N, :] = X
        return Windows(X=Xp[None, ...], t_index=np.array([0], dtype=int))

    idx = []
    for start in range(0, max(1, N - T + 1), H):
        stop = start + T
        if stop <= N:
            idx.append((start, stop))

    W = len(idx)
    if W == 0:
        a = max(0, N - T); b = N
        return Windows(X=X[a:b, :][None, ...], t_index=np.array([0], dtype=int))

    Xw = np.zeros((W, T, C), dtype=float)
    for i, (a, b) in enumerate(idx):
        Xw[i] = X[a:b, :]
    return Windows(X=Xw, t_index=np.arange(W, dtype=int))


## 6) Compute per-window EEWI and fuse to H-EEWI

In [ ]:

@dataclass
class HEewiResult:
    heewi: np.ndarray               # [W]
    heewi_per_band: np.ndarray      # [W, K]
    freqs_example: np.ndarray       # [F]

def compute_heewi_for_windows(wins: Windows, fs: float,
                              bands: List[Tuple[float, float]], alpha: float,
                              fuse_weights: Optional[np.ndarray] = None,
                              isotonic_smooth: bool = True) -> HEewiResult:
    W, T, C = wins.X.shape
    nperseg = min(1024, T)
    K = len(bands)
    heewi_per_band = np.zeros((W, K), dtype=float)
    heewi = np.zeros(W, dtype=float)
    freqs_example = None

    for i in tqdm(range(W), desc="EEWI per window"):
        Xi = wins.X[i]
        freqs, psd_sum = aggregate_multichannel_psd(Xi, fs, nperseg)
        if freqs_example is None:
            freqs_example = freqs
        eewi_k = eewi_from_psd(freqs, psd_sum, bands, alpha)
        heewi_per_band[i] = eewi_k
        heewi[i] = fuse_hierarchical(eewi_k, weights=fuse_weights)

    if isotonic_smooth and W >= 1:
        iso = IsotonicRegression(increasing=True, out_of_bounds="clip")
        heewi = iso.fit_transform(np.arange(W), heewi)

    return HEewiResult(heewi=heewi, heewi_per_band=heewi_per_band, freqs_example=freqs_example)


## 7) Plotting helpers

In [ ]:

def plot_heewi_curve(t_idx: np.ndarray, heewi: np.ndarray, title: str, path: str):
    plt.figure()
    plt.plot(t_idx, heewi)
    plt.xlabel("Window index")
    plt.ylabel("H-EEWI")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(path, dpi=160)
    plt.close()

def plot_rul_with_pi(t_idx: np.ndarray, rul_pred: np.ndarray, rul_true: Optional[np.ndarray],
                     lo: Optional[np.ndarray], hi: Optional[np.ndarray], title: str, path: str):
    plt.figure()
    if lo is not None and hi is not None:
        plt.fill_between(t_idx, lo, hi, alpha=0.3, label="PI")
    plt.plot(t_idx, rul_pred, label="Pred RUL")
    if rul_true is not None:
        plt.plot(t_idx, rul_true, linestyle='--', label="True RUL")
    plt.xlabel("Window index")
    plt.ylabel("RUL (windows)")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(path, dpi=160)
    plt.close()


## 8) Prepare a single cutter → windows → H-EEWI (sanity check)

In [ ]:

def prepare_series_for_cutter(df: pd.DataFrame, fs: float, window_sec: float, hop_sec: float,
                              bands: List[Tuple[float, float]], alpha: float,
                              fuse_weights: Optional[np.ndarray], iso_smooth: bool):
    X = df.to_numpy(dtype=float)
    wins = make_windows(X, fs, window_sec, hop_sec)

    he = compute_heewi_for_windows(wins, fs, bands, alpha,
                                   fuse_weights=fuse_weights, isotonic_smooth=iso_smooth)

    W = len(wins.t_index)
    rul_true = np.arange(W-1, -1, -1)  # proxy if true labels aren't per-sample
    return {
        "heewi": he.heewi,
        "heewi_per_band": he.heewi_per_band,
        "t_idx": wins.t_index,
        "rul_true": rul_true
    }

# === Try a single cutter (first TRAIN) ===
ensure_dir(CFG.outdir)
first_c = CFG.TRAIN_CUTTERS[0]
df_demo = loader.load_cutter(first_c)

print(f"{first_c} shape:", df_demo.shape)
WINS = make_windows(df_demo.to_numpy(dtype=float), CFG.fs, CFG.window_sec, CFG.hop_sec)
print("Windows:", WINS.X.shape)  # [W, T, C]

demo = prepare_series_for_cutter(df_demo, CFG.fs, CFG.window_sec, CFG.hop_sec,
                                 list(CFG.bands), CFG.alpha_energy, None, CFG.isotonic_smooth_heewi)

plot_heewi_curve(demo["t_idx"], demo["heewi"],
                 f"{first_c} H-EEWI (raw, equal weights)",
                 os.path.join(CFG.outdir, f"{first_c}_heewi_curve.png"))

print("Saved:", os.path.join(CFG.outdir, f"{first_c}_heewi_curve.png"))


## 9) Learn simple convex fusion weights over train cutters

In [ ]:

def learn_fuse_weights(series_list: List[Dict]) -> np.ndarray:
    """Grid-search small convex weights to reduce proxy RUL RMSE."""
    K = series_list[0]["heewi_per_band"].shape[1]
    for s in series_list:
        if np.any(~np.isfinite(s["heewi_per_band"])):
            return np.ones(K) / K

    step = 0.25
    ws = np.arange(0, 1+1e-9, step)
    grid = []
    for a in ws:
        for b in ws:
            c = 1 - a - b
            if c < -1e-9: 
                continue
            c = max(0.0, c)
            ssum = a + b + c
            grid.append(np.array([a, b, c]) / (ssum if ssum>0 else 1.0))

    best_rmse = float('inf'); best_w = np.ones(K)/K
    for w in grid:
        rmses = []
        for s in series_list:
            heewi = (s["heewi_per_band"] @ w)
            iso_s = IsotonicRegression(increasing=True, out_of_bounds="clip")
            s_frac = np.linspace(0, 1, len(s["t_idx"]))
            iso_s.fit(heewi, s_frac)
            s_hat = iso_s.predict(heewi)
            rul_hat = (1 - s_hat) * (len(s["t_idx"]) - 1)
            rmses.append(mean_squared_error(s["rul_true"], rul_hat, squared=False))
        rm = float(np.mean(rmses))
        if rm < best_rmse:
            best_rmse, best_w = rm, w
    return best_w

# Build per-cutter series for all TRAIN_CUTTERS (equal weights first)
train_series = []
for c in CFG.TRAIN_CUTTERS:
    df = loader.load_cutter(c)
    s = prepare_series_for_cutter(df, CFG.fs, CFG.window_sec, CFG.hop_sec,
                                  list(CFG.bands), CFG.alpha_energy, None, CFG.isotonic_smooth_heewi)
    train_series.append(s)
print("Prepared", len(train_series), "train cutters.")

# Learn or set fuse weights
if CFG.fuse_weights is None:
    w_fuse = learn_fuse_weights(train_series)
else:
    w_fuse = np.array(CFG.fuse_weights, dtype=float)
w_fuse


## 10) Recompute H-EEWI with learned fusion weights (and smooth)

In [ ]:

for idx, s in enumerate(train_series):
    heewi = (s["heewi_per_band"] @ w_fuse)
    if CFG.isotonic_smooth_heewi:
        iso = IsotonicRegression(increasing=True, out_of_bounds="clip")
        heewi = iso.fit_transform(np.arange(len(heewi)), heewi)
    s["heewi"] = heewi
    c = CFG.TRAIN_CUTTERS[idx]
    plot_heewi_curve(s["t_idx"], s["heewi"], f"{c} H-EEWI (fused)", os.path.join(CFG.outdir, f"{c}_heewi_curve_fused.png"))
    print("Saved:", os.path.join(CFG.outdir, f"{c}_heewi_curve_fused.png"))


## 11) Fit global monotone mappings: H-EEWI→s and s→RUL

In [ ]:

def fit_global_normalization(train_heewi: np.ndarray, train_time_frac: np.ndarray) -> IsotonicRegression:
    iso = IsotonicRegression(increasing=True, y_min=0.0, y_max=1.0, out_of_bounds="clip")
    iso.fit(train_heewi, train_time_frac)
    return iso

def fit_global_rul_mapping(train_s: np.ndarray, train_rul: np.ndarray) -> IsotonicRegression:
    iso_dec = IsotonicRegression(increasing=False, out_of_bounds="clip")
    iso_dec.fit(train_s, train_rul)
    return iso_dec

# Pool training
train_heewi = np.concatenate([s["heewi"] for s in train_series])
train_sfrac = np.concatenate([np.linspace(0, 1, len(s["t_idx"])) for s in train_series])
iso_heewi_to_s = fit_global_normalization(train_heewi, train_sfrac)

train_s = np.concatenate([iso_heewi_to_s.predict(s["heewi"]) for s in train_series])
train_rul = np.concatenate([s["rul_true"] for s in train_series])
iso_s_to_rul = fit_global_rul_mapping(train_s, train_rul)

print("Fitted global monotone mappings.")


## 12) LOTO evaluation on train cutters

In [ ]:

@dataclass
class Metrics:
    rmse: float
    mae: float
    monotonicity: float  # fraction of non-decreasing pairs in H-EEWI

def monotone_fraction(x: np.ndarray) -> float:
    diffs = np.diff(x)
    return float(np.mean(diffs >= -1e-9))

def eval_fold(y_true_rul: np.ndarray, y_pred_rul: np.ndarray, heewi_curve: np.ndarray) -> Metrics:
    rmse = math.sqrt(mean_squared_error(y_true_rul, y_pred_rul))
    mae = mean_absolute_error(y_true_rul, y_pred_rul)
    mono = monotone_fraction(heewi_curve)
    return Metrics(rmse=rmse, mae=mae, monotonicity=mono)

fold_results = {}
for heldout_idx, heldout in enumerate(CFG.TRAIN_CUTTERS):
    # Train on others
    train_tools = [i for i,c in enumerate(CFG.TRAIN_CUTTERS) if c != heldout]
    pool_heewi = np.concatenate([train_series[i]["heewi"] for i in train_tools])
    pool_sfrac = np.concatenate([np.linspace(0, 1, len(train_series[i]["t_idx"])) for i in train_tools])

    iso_h2s = fit_global_normalization(pool_heewi, pool_sfrac)
    s_hat_train = iso_h2s.predict(pool_heewi)

    pool_s_all = np.concatenate([iso_h2s.predict(train_series[i]["heewi"]) for i in train_tools])
    pool_rul_all = np.concatenate([train_series[i]["rul_true"] for i in train_tools])
    iso_s2r = fit_global_rul_mapping(pool_s_all, pool_rul_all)

    # Eval on heldout
    s_ho = train_series[heldout_idx]
    s_seq = iso_h2s.predict(s_ho["heewi"])
    rul_pred = iso_s2r.predict(s_seq)
    metrics = eval_fold(s_ho["rul_true"], rul_pred, s_ho["heewi"])

    # Plot
    plot_rul_with_pi(s_ho["t_idx"], rul_pred, s_ho["rul_true"], None, None,
                     title=f"{heldout} RUL (LOTO)", path=os.path.join(CFG.outdir, f"{heldout}_rul_loto.png"))

    fold_results[heldout] = {"metrics": asdict(metrics), "fuse_weights": w_fuse.tolist()}
    print(heldout, metrics)

summary = {
    "fuse_weights": w_fuse.tolist(),
    "folds": fold_results,
    "RMSE_mean": float(np.mean([fold_results[c]["metrics"]["rmse"] for c in fold_results])),
    "MAE_mean": float(np.mean([fold_results[c]["metrics"]["mae"] for c in fold_results])),
    "MonotoneFrac_mean": float(np.mean([fold_results[c]["metrics"]["monotonicity"] for c in fold_results]))
}
ensure_dir(CFG.outdir)
with open(os.path.join(CFG.outdir, "summary.json"), "w") as f:
    json.dump(summary, f, indent=2)
print("Saved:", os.path.join(CFG.outdir, "summary.json"))
summary


## 13) Bootstrap utilities (entropy-level & window-level)

In [ ]:

@dataclass
class BootstrapResult:
    rul_lo: np.ndarray
    rul_hi: np.ndarray
    rul_samples: Optional[np.ndarray] = None

def entropy_bootstrap_heewi(wins: Windows, fs: float, bands: List[Tuple[float, float]], alpha: float,
                            fuse_weights: Optional[np.ndarray], B: int, sigma: float,
                            heewi_iso_smooth: bool = True) -> np.ndarray:
    W, T, C = wins.X.shape
    nperseg = min(1024, T)
    heewi_samples = np.zeros((B, W), dtype=float)

    # Precompute PSD caches
    cache = []
    for i in range(W):
        freqs, psd_sum = aggregate_multichannel_psd(wins.X[i], fs, nperseg)
        cache.append((freqs, psd_sum))

    for b in tqdm(range(B), desc="Entropy bootstrap"):
        heewi_b = np.zeros(W, dtype=float)
        for i in range(W):
            freqs, psd_sum = cache[i]
            noise = np.exp(np.random.normal(loc=0.0, scale=sigma, size=psd_sum.shape))
            psd_noisy = psd_sum * noise
            eewi_k = eewi_from_psd(freqs, psd_noisy, bands, alpha)
            heewi_b[i] = fuse_hierarchical(eewi_k, weights=fuse_weights)
        if heewi_iso_smooth and W >= 1:
            iso = IsotonicRegression(increasing=True, out_of_bounds="clip")
            heewi_b = iso.fit_transform(np.arange(W), heewi_b)
        heewi_samples[b] = heewi_b
    return heewi_samples

def window_bootstrap_rul(s_seq: np.ndarray, rul_seq: np.ndarray, B: int,
                         q_lo: float = 0.05, q_hi: float = 0.95) -> BootstrapResult:
    N = len(s_seq)
    rul_samples = np.zeros((B, N), dtype=float)
    idx_all = np.arange(N)
    for b in range(B):
        idx = np.random.choice(idx_all, size=N, replace=True)
        iso_b = IsotonicRegression(increasing=False, out_of_bounds="clip")
        iso_b.fit(s_seq[idx], rul_seq[idx])
        rul_samples[b] = iso_b.predict(s_seq)
    lo = np.quantile(rul_samples, q_lo, axis=0)
    hi = np.quantile(rul_samples, q_hi, axis=0)
    return BootstrapResult(rul_lo=lo, rul_hi=hi, rul_samples=None)


## 14) Train on all train cutters, infer on TEST_CUTTERS (with PI)

In [ ]:

@dataclass
class TrainedGlobal:
    iso_heewi_to_s: IsotonicRegression
    iso_s_to_rul: IsotonicRegression
    fuse_weights: np.ndarray

def train_global_on_all_train(cfg: Config, channel_cols: Optional[List[str]] = None) -> TrainedGlobal:
    ensure_dir(cfg.outdir)
    loader = PHM2010Loader(cfg.BASE, channel_cols=channel_cols)

    # Build train series
    series = []
    for c in cfg.TRAIN_CUTTERS:
        df = loader.load_cutter(c)
        s = prepare_series_for_cutter(df, cfg.fs, cfg.window_sec, cfg.hop_sec,
                                      list(cfg.bands), cfg.alpha_energy, None, cfg.isotonic_smooth_heewi)
        series.append(s)

    w_fuse = learn_fuse_weights(series) if cfg.fuse_weights is None else np.array(cfg.fuse_weights)

    # Recompute fused curves
    for s in series:
        heewi = (s["heewi_per_band"] @ w_fuse)
        if cfg.isotonic_smooth_heewi:
            iso = IsotonicRegression(increasing=True, out_of_bounds="clip")
            heewi = iso.fit_transform(np.arange(len(heewi)), heewi)
        s["heewi"] = heewi

    # Fit global mappings
    train_heewi = np.concatenate([s["heewi"] for s in series])
    train_sfrac = np.concatenate([np.linspace(0, 1, len(s["t_idx"])) for s in series])
    iso_heewi_to_s = fit_global_normalization(train_heewi, train_sfrac)

    train_s = np.concatenate([iso_heewi_to_s.predict(s["heewi"]) for s in series])
    train_rul = np.concatenate([s["rul_true"] for s in series])
    iso_s_to_rul = fit_global_rul_mapping(train_s, train_rul)

    return TrainedGlobal(iso_heewi_to_s=iso_heewi_to_s, iso_s_to_rul=iso_s_to_rul, fuse_weights=w_fuse)

def predict_for_test_cutters(cfg: Config, tg: TrainedGlobal, channel_cols: Optional[List[str]] = None):
    ensure_dir(cfg.outdir)
    loader = PHM2010Loader(cfg.BASE, channel_cols=channel_cols)

    for c in cfg.TEST_CUTTERS:
        df = loader.load_cutter(c)
        X = df.to_numpy(dtype=float)
        wins = make_windows(X, cfg.fs, cfg.window_sec, cfg.hop_sec)
        he = compute_heewi_for_windows(wins, cfg.fs, list(cfg.bands), cfg.alpha_energy,
                                       fuse_weights=tg.fuse_weights, isotonic_smooth=cfg.isotonic_smooth_heewi)
        s_seq = tg.iso_heewi_to_s.predict(he.heewi)
        rul_pred = tg.iso_s_to_rul.predict(s_seq)

        # Entropy-bootstrap at EEWI level, propagate to RUL
        heewi_samples = entropy_bootstrap_heewi(
            wins=wins, fs=cfg.fs, bands=list(cfg.bands), alpha=cfg.alpha_energy,
            fuse_weights=tg.fuse_weights, B=cfg.bootstrap_B,
            sigma=cfg.entropy_bootstrap_sigma, heewi_iso_smooth=cfg.isotonic_smooth_heewi
        )
        rul_samples = []
        for b in range(heewi_samples.shape[0]):
            s_b = tg.iso_heewi_to_s.predict(heewi_samples[b])
            rul_b = tg.iso_s_to_rul.predict(s_b)
            rul_samples.append(rul_b)
        rul_samples = np.stack(rul_samples, axis=0)
        lo = np.quantile(rul_samples, 0.05, axis=0)
        hi = np.quantile(rul_samples, 0.95, axis=0)

        plot_rul_with_pi(wins.t_index, rul_pred, None, lo, hi,
                         title=f"{c} RUL (inference)",
                         path=os.path.join(cfg.outdir, f"{c}_rul_infer.png"))

        # Save arrays
        np.save(os.path.join(cfg.outdir, f"{c}_rul_pred.npy"), rul_pred)
        np.save(os.path.join(cfg.outdir, f"{c}_rul_pi_lo.npy"), lo)
        np.save(os.path.join(cfg.outdir, f"{c}_rul_pi_hi.npy"), hi)
        print(f"Saved predictions & PI for {c}.")

# === Train+infer ===
tg = train_global_on_all_train(CFG, channel_cols=None)
predict_for_test_cutters(CFG, tg, channel_cols=None)


## 15) (Optional) End-to-end helpers

In [ ]:

def run_loto_experiments(cfg: Config, channel_cols: Optional[List[str]] = None):
    ensure_dir(cfg.outdir)
    loader = PHM2010Loader(cfg.BASE, channel_cols=channel_cols)

    # Build series
    series_by_cutter: Dict[str, Dict] = {}
    for c in cfg.TRAIN_CUTTERS:
        df = loader.load_cutter(c)
        s = prepare_series_for_cutter(df, cfg.fs, cfg.window_sec, cfg.hop_sec,
                                      list(cfg.bands), cfg.alpha_energy, None, cfg.isotonic_smooth_heewi)
        series_by_cutter[c] = s
        plot_heewi_curve(s["t_idx"], s["heewi"], f"{c} H-EEWI (raw, equal weights)",
                         os.path.join(cfg.outdir, f"{c}_heewi_curve.png"))

    # Learn or set fusion weights
    if cfg.fuse_weights is None:
        w_fuse = learn_fuse_weights(list(series_by_cutter.values()))
    else:
        w_fuse = np.array(cfg.fuse_weights, dtype=float)

    # Recompute H-EEWI using learned weights
    for c in cfg.TRAIN_CUTTERS:
        s = series_by_cutter[c]
        heewi = (s["heewi_per_band"] @ w_fuse)
        if cfg.isotonic_smooth_heewi:
            iso = IsotonicRegression(increasing=True, out_of_bounds="clip")
            heewi = iso.fit_transform(np.arange(len(heewi)), heewi)
        s["heewi"] = heewi
        plot_heewi_curve(s["t_idx"], s["heewi"], f"{c} H-EEWI (fused)",
                         os.path.join(cfg.outdir, f"{c}_heewi_curve_fused.png"))

    # LOTO folds
    fold_metrics = []
    all_results = {}
    for heldout in cfg.TRAIN_CUTTERS:
        train_tools = [c for c in cfg.TRAIN_CUTTERS if c != heldout]

        train_heewi = []
        train_sfrac = []
        train_s_to_rul_s = []
        train_rul = []
        for c in train_tools:
            s = series_by_cutter[c]
            W = len(s["t_idx"])
            s_frac = np.linspace(0, 1, W)
            train_heewi.append(s["heewi"])
            train_sfrac.append(s_frac)
            train_s_to_rul_s.append(s_frac)
            train_rul.append(s["rul_true"])

        train_heewi = np.concatenate(train_heewi)
        train_sfrac = np.concatenate(train_sfrac)
        train_s_to_rul_s = np.concatenate(train_s_to_rul_s)
        train_rul = np.concatenate(train_rul)

        iso_heewi_to_s = fit_global_normalization(train_heewi, train_sfrac)
        s_hat_train = iso_heewi_to_s.predict(train_heewi)
        iso_s_to_rul = fit_global_rul_mapping(train_s_to_rul_s, train_rul)

        s_ho = series_by_cutter[heldout]
        s_seq = iso_heewi_to_s.predict(s_ho["heewi"])
        rul_pred = iso_s_to_rul.predict(s_seq)

        # Metrics
        rmse = math.sqrt(mean_squared_error(s_ho["rul_true"], rul_pred))
        mae = mean_absolute_error(s_ho["rul_true"], rul_pred)
        mono = float(np.mean(np.diff(s_ho["heewi"]) >= -1e-9))

        # PIs (window-level bootstrap)
        if cfg.window_bootstrap:
            pi = window_bootstrap_rul(s_seq, rul_pred, B=cfg.bootstrap_B)
            lo, hi = pi.rul_lo, pi.rul_hi
        else:
            lo, hi = None, None

        plot_rul_with_pi(s_ho["t_idx"], rul_pred, s_ho["rul_true"], lo, hi,
                         title=f"{heldout} RUL (LOTO)",
                         path=os.path.join(cfg.outdir, f"{heldout}_rul_loto.png"))

        all_results[heldout] = {
            "metrics": {"rmse": rmse, "mae": mae, "monotonicity": mono},
            "fuse_weights": w_fuse.tolist()
        }
        fold_metrics.append((rmse, mae, mono))

    agg = {
        "fuse_weights": w_fuse.tolist(),
        "folds": all_results,
        "RMSE_mean": float(np.mean([m[0] for m in fold_metrics])),
        "MAE_mean": float(np.mean([m[1] for m in fold_metrics])),
        "MonotoneFrac_mean": float(np.mean([m[2] for m in fold_metrics]))
    }
    with open(os.path.join(cfg.outdir, "summary.json"), "w") as f:
        json.dump(agg, f, indent=2)
    print("=== LOTO Summary ===\n", json.dumps(agg, indent=2))

# (Optional) run:
# run_loto_experiments(CFG, channel_cols=None)
